In [10]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn import linear_model as lm
from sklearn.metrics import mean_squared_error

original_data = pd.read_excel("Original_Data.xlsx", header = 3, usecols = "A:CC")
hull_data = pd.read_csv('Houseprice_data_scaled.csv') 

lr = lm.LinearRegression()
rg = lm.Ridge(alpha = 0.01)
ls = lm.Lasso(alpha = 0.01)

scaler = StandardScaler()


In [11]:
#Imputation Strategy X: Remove all missing data (Scrapped)

# imputed_data_1 = original_data.copy()

# nan_map = imputed_data_1.isna()

# for i, row in nan_map.iterrows():
#     if row['LotFrontage']:
#         imputed_data_1 = imputed_data_1.drop([i])

# X = imputed_data_1.loc[:, ["LotFrontage"]]
# y = np.dot(X, np.array(imputed_data_1.loc[:, ['SalePrice']]))

# coeffs = pd.DataFrame(
#     [
#         ['intercept'] + list(X_train.columns),
#         list(lr.intercept_) + list(lr.coef_[0])
#     ]
# ).transpose().set_index(0)

# strat1_linear = lm.LinearRegression().fit(X, y)
# strat1_lasso = lm.Lasso(alpha = 0.01).fit(X, y)
# strat1_ridge = lm.Ridge(alpha = 0.01).fit(X, y)

# print(strat1_linear.score(X, y))
# print(strat1_lasso.score(X, y))
# print(strat1_ridge.score(X, y))

In [12]:
#Scaling the data

In [13]:
#Finding Optimal Alphas
def AlphaOptimizer(method, X_train, y_train, X_val, y_val):
    
    mses=[]
    
    if method == "Ridge":
        alphas=[0.01*1800, 0.02*1800, 0.03*1800, 0.04*1800, 0.05*1800, 0.075*1800,0.1*1800,0.2*1800, 0.4*1800]
        for alpha in alphas:
            ridge = lm.Ridge(alpha=alpha)
            ridge.fit(X_train,y_train)
            pred = ridge.predict(X_val)
            mses.append(mean_squared_error(y_val,pred))
            return min(mses)
        
    elif method == "Lasso":
        alphas=[0.01/2, 0.02/2, 0.03/2, 0.04/2, 0.05/2, 0.075/2, 0.1/2]
        for alpha in alphas:
            lasso = lm.Lasso(alpha=alpha)
            lasso.fit(X_train,y_train)
            pred = lasso.predict(X_val)
            mses.append(mean_squared_error(y_val,pred))
            return min(mses)
        
    else:
        return("Not available method")

In [14]:
#Imputation Strategy 1: Replace missing data with the mean of data
imputed_data_1 = original_data.copy()

nan_map = imputed_data_1.isna()

mean = imputed_data_1["LotFrontage"].mean()

for i, row in nan_map.iterrows():
    if row['LotFrontage']:
        imputed_data_1["LotFrontage"][i] = mean

train1 = hull_data[:1800]

train1.insert(0, "LotFrontage", imputed_data_1.loc[:1799, ["LotFrontage"]].values)

val1 = hull_data[1800:2400]

val1.insert(0, "LotFrontage", imputed_data_1.loc[1800:2399, ["LotFrontage"]].values)

X_train1, X_val1 = train1.drop('Sale Price', axis=1), val1.drop('Sale Price', axis=1)
y_train1, y_val1 = train1[['Sale Price']], val1[['Sale Price']] 

X_train1 = scaler.fit_transform(X_train1)
X_val1 = scaler.transform(X_val1)

#Linear Regression
strat1_linear = lr.fit(X_train1, y_train1)
val1_lr_pred = strat1_linear.predict(X_val1)
print("Linear Regression:")
print("R-Score:", strat1_linear.score(X_train1, y_train1))
print("MSE:", mean_squared_error(y_val1, val1_lr_pred))

#Ridge
strat1_ridge = rg.fit(X_train1, y_train1)
val1_rg_pred = strat1_ridge.predict(X_val1)
print("\nRidge:")
print("R-Score:", strat1_ridge.score(X_train1, y_train1))
print("MSE:", AlphaOptimizer("Ridge", X_train1, y_train1, X_val1, y_val1))

#Lasso
strat1_lasso = ls.fit(X_train1, y_train1)
val1_ls_pred = strat1_lasso.predict(X_val1)
print("\nLasso:")
print("R-Score:", strat1_lasso.score(X_train1, y_train1))
print("MSE:", AlphaOptimizer("Lasso", X_train1, y_train1, X_val1, y_val1))

/var/folders/z8/5wdvzpnn5msc6zhhnq9ztx580000gn/T/ipykernel_5675/1522365498.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  imputed_data_1["LotFrontage"][i] = mean
/var/folders/z8/5wdvzpnn5msc6zhhnq9ztx580000gn/T/ipykernel_5675/1522365498.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  imputed_data_1["LotFrontage"][i] = mean
/var/folders/z8/5wdvzpnn5msc6zhhnq9ztx580000gn/T/ipykernel_5675/1522365498.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returnin

Linear Regression:
R-Score: 0.8878587627685235
MSE: 0.11325046944616939

Ridge:
R-Score: 0.8878587627288108
MSE: 0.11326369106126818

Lasso:
R-Score: 0.8844910428276486
MSE: 0.11282838839994151


In [15]:
#Imputation Strategy 2: Replace missing data with the median of data
imputed_data_2 = original_data.copy()

nan_map = imputed_data_2.isna()

median = imputed_data_2["LotFrontage"].median()

for i, row in nan_map.iterrows():
    if row['LotFrontage']:
        imputed_data_2["LotFrontage"][i] = median

train2 = hull_data[:1800].copy()
val2 = hull_data[1800:2400].copy()

scaled_train_data = scaler.fit_transform(imputed_data_2.loc[:1799, ["LotFrontage"]])
scaled_val_data = scaler.fit_transform(imputed_data_2.loc[1800:2399, ["LotFrontage"]])

train2.insert(0, "LotFrontage", scaled_train_data)

val2.insert(0, "LotFrontage", scaled_val_data)

X_train2, X_val2 = train2.drop('Sale Price', axis=1), val2.drop('Sale Price', axis=1)
y_train2, y_val2 = train2[['Sale Price']], val2[['Sale Price']] 

X_train2 = scaler.fit_transform(X_train2)
X_val2 = scaler.transform(X_val2)

#Linear Regression
strat2_linear = lr.fit(X_train2, y_train2)
val2_lr_pred = strat2_linear.predict(X_val2)
print("Linear Regression:")
print("R-Score:", strat2_linear.score(X_train2, y_train2))
print("MSE:", mean_squared_error(y_val2, val2_lr_pred))

#Ridge
strat2_ridge = rg.fit(X_train2, y_train2)
val2_rg_pred = strat2_ridge.predict(X_val2)
print("\nRidge:")
print("R-Score:", strat2_ridge.score(X_train2, y_train2))
print("MSE:", AlphaOptimizer("Ridge", X_train2, y_train2, X_val2, y_val2))

#Lasso
strat2_lasso = ls.fit(X_train2, y_train2)
val2_ls_pred = strat2_lasso.predict(X_val2)
print("\nLasso:")
print("R-Score:", strat2_lasso.score(X_train2, y_train2))
print("MSE:", AlphaOptimizer("Lasso", X_train2, y_train2, X_val2, y_val2))

/var/folders/z8/5wdvzpnn5msc6zhhnq9ztx580000gn/T/ipykernel_5675/4288613964.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  imputed_data_2["LotFrontage"][i] = median
/var/folders/z8/5wdvzpnn5msc6zhhnq9ztx580000gn/T/ipykernel_5675/4288613964.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  imputed_data_2["LotFrontage"][i] = median
/var/folders/z8/5wdvzpnn5msc6zhhnq9ztx580000gn/T/ipykernel_5675/4288613964.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#retu

Linear Regression:
R-Score: 0.887885687834035
MSE: 0.11312981468437366

Ridge:
R-Score: 0.8878856877940827
MSE: 0.1131429394936158

Lasso:
R-Score: 0.8845191770319231
MSE: 0.11269411665008434


In [16]:
#Imputation Strategy 3: Replace missing data with linear interpolation prediction
imputed_data_3 = original_data.copy()

nan_map = imputed_data_3.isna()

last_valid_value = imputed_data_3["LotFrontage"][imputed_data_3["LotFrontage"].first_valid_index()]

for i, row in nan_map.iterrows():
    if row['LotFrontage']:
        j = i
        while nan_map["LotFrontage"][j]:
            j += 1
        next_valid_value = imputed_data_3["LotFrontage"][j]
        imputed_data_3["LotFrontage"][i] = (next_valid_value + last_valid_value) / 2
    else:
        last_valid_value = imputed_data_3["LotFrontage"][i]

train3 = hull_data[:1800]

train3.insert(0, "LotFrontage", imputed_data_3.loc[:1799, ["LotFrontage"]].values)

val3 = hull_data[1800:2400]

val3.insert(0, "LotFrontage", imputed_data_3.loc[1800:2399, ["LotFrontage"]].values)

X_train3, X_val3 = train3.drop('Sale Price', axis=1), val3.drop('Sale Price', axis=1)
y_train3, y_val3 = train3[['Sale Price']], val3[['Sale Price']] 

X_train3 = scaler.fit_transform(X_train3)
X_val3 = scaler.transform(X_val3)

#Linear Regression
strat3_linear = lr.fit(X_train3, y_train3)
val3_lr_pred = strat3_linear.predict(X_val3)
print("Linear Regression:")
print("R-Score:", strat3_linear.score(X_train3, y_train3))
print("MSE:", mean_squared_error(y_val3, val3_lr_pred))

#Ridge
strat3_ridge = rg.fit(X_train3, y_train3)
val3_rg_pred = strat3_ridge.predict(X_val3)
print("\nRidge:")
print("R-Score:", strat3_ridge.score(X_train3, y_train3))
print("MSE:", AlphaOptimizer("Ridge", X_train3, y_train3, X_val3, y_val3))

#Lasso
strat3_lasso = ls.fit(X_train3, y_train3)
val3_ls_pred = strat3_lasso.predict(X_val3)
print("\nLasso:")
print("R-Score:", strat3_lasso.score(X_train3, y_train3))
print("MSE:", AlphaOptimizer("Lasso", X_train3, y_train3, X_val3, y_val3))

/var/folders/z8/5wdvzpnn5msc6zhhnq9ztx580000gn/T/ipykernel_5675/738514822.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  imputed_data_3["LotFrontage"][i] = (next_valid_value + last_valid_value) / 2
/var/folders/z8/5wdvzpnn5msc6zhhnq9ztx580000gn/T/ipykernel_5675/738514822.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  imputed_data_3["LotFrontage"][i] = (next_valid_value + last_valid_value) / 2
/var/folders/z8/5wdvzpnn5msc6zhhnq9ztx580000gn/T/ipykernel_5675/738514822.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https:/

Linear Regression:
R-Score: 0.887542069212596
MSE: 0.1136372553103597

Ridge:
R-Score: 0.8875437620895418
MSE: 0.11364028204620207

Lasso:
R-Score: 0.8841481871317547
MSE: 0.11321248417972674


/var/folders/z8/5wdvzpnn5msc6zhhnq9ztx580000gn/T/ipykernel_5675/738514822.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  imputed_data_3["LotFrontage"][i] = (next_valid_value + last_valid_value) / 2
/var/folders/z8/5wdvzpnn5msc6zhhnq9ztx580000gn/T/ipykernel_5675/738514822.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  imputed_data_3["LotFrontage"][i] = (next_valid_value + last_valid_value) / 2
/var/folders/z8/5wdvzpnn5msc6zhhnq9ztx580000gn/T/ipykernel_5675/738514822.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https:/

In [17]:
#

scaled_data = scaler.fit_transform(imputed_data_2.loc[:, ["LotFrontage"]])

data_a = hull_data.copy()
print(data_a)
data_a.insert(0, "LotFrontage", scaled_data)

print(scaled_data)
print(data_a)

       LotArea  OverallQual  OverallCond  YearBuilt  YearRemodAdd  BsmtFinSF1  \
0    -0.199572     0.652747    -0.512407   1.038851      0.875754    0.597837   
1    -0.072005    -0.072527     2.189741   0.136810     -0.432225    1.218528   
2     0.111026     0.652747    -0.512407   0.972033      0.827310    0.095808   
3    -0.077551     0.652747    -0.512407  -1.901135     -0.722887   -0.520319   
4     0.444919     1.378022    -0.512407   0.938624      0.730423    0.481458   
...        ...          ...          ...        ...           ...         ...   
2903 -0.922157    -1.523077     1.289025  -0.063644     -0.722887   -1.013221   
2904 -0.926816    -1.523077    -0.512407  -0.063644     -0.722887   -0.438169   
2905  1.081646    -0.797802     1.289025  -0.397733      0.536648    1.779888   
2906  0.021285    -0.797802    -0.512407   0.671352      0.342874   -0.244203   
2907 -0.069010     0.652747    -0.512407   0.704761      0.439761    0.716499   

      BsmtUnfSF  TotalBsmtS

### Part B: Adding Lot Shape

Lot Shape has 4 Categories. 
- let 0 = regular lot shape
- let 1 = slightly irregular
- let 2 = moderately irregular
- let 3 = irregular

### Create Categorical Variable

In [18]:
lot_shapes = []
for row in original_data['LotShape']:
    if row == 'Reg':
        lot_shapes.append(0)
    elif row == 'IR1':
        lot_shapes.append(1)
    elif row == 'IR2':
        lot_shapes.append(2)
    else:
        lot_shapes.append(3)

Lot_Shape = pd.DataFrame(lot_shapes, columns=['LotShape'])

print(Lot_Shape)

      LotShape
0            0
1            0
2            1
3            1
4            1
...        ...
2903         0
2904         0
2905         0
2906         0
2907         0

[2908 rows x 1 columns]


### Standardizing Dummies

In [19]:
LotShape_standardized = Lot_Shape
LotShape_standardized = (LotShape_standardized - LotShape_standardized.mean()) 
LotShape_standardized.head(5)


,LotShape
0,-0.398556
1,-0.398556
2,0.601444
3,0.601444
4,0.601444


### Adding LotShape to DataFrame

In [20]:
data_b = pd.concat([data_a, LotShape_standardized], axis=1)
data_b.head()

,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,BsmtFinSF1,BsmtUnfSF,TotalBsmtSF,1stFlrSF,...,SWISU,Sawyer,SawyerW,Somerst,StoneBr,Timber,Veenker,Bsmt Qual,Sale Price,LotShape
0,-0.188463,-0.199572,0.652747,-0.512407,1.038851,0.875754,0.597837,-0.937245,-0.482464,-0.808820,...,-0.136621,-0.2253,-0.214192,-0.268378,-0.127929,-0.152629,-0.091644,0.584308,0.358489,-0.398556
1,0.535895,-0.072005,-0.072527,2.189741,0.136810,-0.432225,1.218528,-0.635042,0.490326,0.276358,...,-0.136621,-0.2253,-0.214192,-0.268378,-0.127929,-0.152629,10.905682,0.584308,0.008849,-0.398556
2,-0.043591,0.111026,0.652747,-0.512407,0.972033,0.827310,0.095808,-0.296754,-0.329118,-0.637758,...,-0.136621,-0.2253,-0.214192,-0.268378,-0.127929,-0.152629,-0.091644,0.584308,0.552733,0.601444
3,-0.429915,-0.077551,0.652747,-0.512407,-1.901135,-0.722887,-0.520319,-0.057698,-0.722067,-0.528171,...,-0.136621,-0.2253,-0.214192,-0.268378,-0.127929,-0.152629,-0.091644,-0.577852,-0.528560,0.601444
4,0.729057,0.444919,1.378022,-0.512407,0.938624,0.730423,0.481458,-0.170461,0.209990,-0.036366,...,-0.136621,-0.2253,-0.214192,-0.268378,-0.127929,-0.152629,-0.091644,0.584308,0.895898,0.601444


### Linear Regression

In [22]:
train = data_b.iloc[:1800]
validation = data_b.iloc[1800:2400]
test = data_b.iloc[2400:2908]

X_train = train.drop('Sale Price', axis=1)
X_val = validation.drop('Sale Price', axis=1)
Y_train, Y_val = train[['Sale Price']], validation[['Sale Price']]

In [23]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

linear_reg = LinearRegression()
linear_reg.fit(X_train, Y_train)
prediction = linear_reg.predict(X_train)
mse = mean_squared_error(Y_train, prediction)
print("MSE: "+ str(mse))
print("RMSE: " + str(np.sqrt(mse)))
print("MAE: " + str(mean_absolute_error(Y_train, prediction)))
prediction1 = linear_reg.predict(X_val)
mse1 = mean_squared_error(Y_train, prediction)
print(mse1)
print("RMSE: " + str(np.sqrt(mse1)))
print("MAE: " + str(mean_absolute_error(Y_train, prediction)))

MSE: 0.11198716827027992
RMSE: 0.33464483900141045
MAE: 0.2292138320305347
0.11198716827027992
RMSE: 0.33464483900141045
MAE: 0.2292138320305347


In [24]:
coeffs = pd.DataFrame(
    [
        ['intercept'] + list(X_train.columns),
        list(linear_reg.intercept_) + list(linear_reg.coef_[0])
    ]
).transpose().set_index(0)
coeffs

,1
0,
intercept,-0.000808
LotFrontage,0.055445
LotArea,0.069429
OverallQual,0.212792
OverallCond,0.097124
YearBuilt,0.164766
YearRemodAdd,0.027101
BsmtFinSF1,0.092344
BsmtUnfSF,-0.033028


### Lasso Regression, alpha = 0.01

In [26]:
%%sql
lasso_reg = Lasso(alpha(0.01))
lasso_reg.fit(X_train, Y_train)

          coeffs = pd.DataFrame(
    [
        ['intercept'] + list(X_train.columns),
        list(lasso_reg.intercept_) + list(lasso_reg.coef_[0])
    ]
).transpose().set_index(0)
coeffs

Exception: Data Source is not selected

In [27]:
prediction = lasso_reg.predict(X_train)
mse = mse(Y_train, prediction)
print("MSE: "+ str(mse))
print("RMSE: " + str(np.sqrt(mse)))
print("MAE: " + str(mean_absolute_error(Y_train, prediction)))
prediction1 = lasso_reg.predict(X_val)
mse1 = mse(Y_train, prediction)
print(mse1)
print("RMSE: " + str(np.sqrt(mse1)))
print("MAE: " + str(mean_absolute_error(Y_train, prediction)))

NameError: name 'lasso_reg' is not defined

### Ridge Regression

In [28]:
from sklearn.linear_model import Ridge

alpha = 1800*0.01
ride_reg = Ridge(alpha=alpha)
ridge_reg.fit(X_train, Y_train)
prediction = ridge_reg.predict(X_train)
mse = mse(Y_train, prediction)
print("MSE: "+ str(mse))
print("RMSE: " + str(np.sqrt(mse)))
print("MAE: " + str(mean_absolute_error(Y_train, prediction)))
prediction1 = ridge_reg.predict(X_val)
mse1 = mse(Y_train, prediction)
print(mse1)
print("RMSE: " + str(np.sqrt(mse1)))
print("MAE: " + str(mean_absolute_error(Y_train, prediction)))

NameError: name 'Ridge' is not defined